# Kapitola 33: Konvoluce - Jak se AI naucila "videt"

V teto kapitole se naucime, jak pocitace "vidi" obrazky pomoci **konvoluce** - zakladni operace, ktera pohani cele moderni pocitacove videni.

## Co se naucime:
- Co je konvoluce a jak funguje
- Filtry (kernely) pro detekci hran, rozmazani, zaostreni
- Praktickou aplikaci konvoluce na obrazky
- Uvod do konvolucnich neuonovych siti (CNN)

---

## 1. Instalace knihoven

Pro praci s obrazky potrebujeme nektere knihovny:

In [ ]:
# Instalace potrebnych knihoven
!pip install numpy matplotlib scikit-image scipy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from skimage.color import rgb2gray
from scipy.signal import convolve2d

# Nastaveni pro hezci grafy
plt.rcParams['figure.figsize'] = [12, 8]
print("Knihovny uspesne nacteny!")

---

## 2. Co je konvoluce? Lupa na vzory

Konvoluce je matematicka operace, kde malou matici cisel (**filtr** nebo **kernel**) "posuvame" pres velky obrazek.

V kazdem kroku:
1. Prekryjeme cast obrazku nasim filtrem
2. Vynasobime odpovidajici si cisla (pixel filtru * pixel obrazku)
3. Vsechny tyto souciny secteme
4. Vysledne jedine cislo zapiseme do noveho obrazku (tzv. **mapa priznaku**)

### Jednoduchy priklad - detekce svisle cary

In [ ]:
# Jednoduchy obrazek 5x5 se svislou carou uprostred
obrazek = np.array([
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0]
])

# Filtr pro detekci svisle hrany (Sobel-like)
filtr_svisle_hrany = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

# Vizualizace
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].imshow(obrazek, cmap='gray')
axes[0].set_title('Puvodni obrazek (svisle cara)', fontsize=14)
for i in range(5):
    for j in range(5):
        axes[0].text(j, i, str(obrazek[i, j]), ha='center', va='center', color='red', fontsize=12)

axes[1].imshow(filtr_svisle_hrany, cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_title('Filtr (detektor svisle hrany)', fontsize=14)
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, str(filtr_svisle_hrany[i, j]), ha='center', va='center', fontsize=12)

plt.tight_layout()
plt.show()

### Provedeni konvoluce rucne

Podivejme se, jak konvoluce funguje krok za krokem:

In [ ]:
# Rucni vypocet konvoluce
def konvoluce_rucne(obrazek, filtr):
    """Jednoducha implementace konvoluce pro pochopeni principu."""
    vyska_obr, sirka_obr = obrazek.shape
    vyska_flt, sirka_flt = filtr.shape
    
    # Velikost vystupu (bez paddingu)
    vyska_out = vyska_obr - vyska_flt + 1
    sirka_out = sirka_obr - sirka_flt + 1
    
    vystup = np.zeros((vyska_out, sirka_out))
    
    for i in range(vyska_out):
        for j in range(sirka_out):
            # Vyber regionu z obrazku
            region = obrazek[i:i+vyska_flt, j:j+sirka_flt]
            # Nasobeni a secteni
            vystup[i, j] = np.sum(region * filtr)
    
    return vystup

# Provedeni konvoluce
vysledek = konvoluce_rucne(obrazek, filtr_svisle_hrany)

print("Puvodni obrazek (5x5):")
print(obrazek)
print("\nFiltr (3x3):")
print(filtr_svisle_hrany)
print("\nVysledek konvoluce (3x3):")
print(vysledek)

In [ ]:
# Vizualizace procesu konvoluce
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Puvodni obrazek
axes[0].imshow(obrazek, cmap='gray')
axes[0].set_title('Puvodni obrazek', fontsize=14)
for i in range(5):
    for j in range(5):
        axes[0].text(j, i, str(obrazek[i, j]), ha='center', va='center', color='red', fontsize=10)

# Filtr
axes[1].imshow(filtr_svisle_hrany, cmap='RdBu', vmin=-1, vmax=1)
axes[1].set_title('Filtr', fontsize=14)
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, str(filtr_svisle_hrany[i, j]), ha='center', va='center', fontsize=10)

# Vysledek
im = axes[2].imshow(vysledek, cmap='RdBu')
axes[2].set_title('Mapa priznaku (vysledek)', fontsize=14)
for i in range(3):
    for j in range(3):
        axes[2].text(j, i, f'{vysledek[i, j]:.0f}', ha='center', va='center', fontsize=10)
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.show()

print("\n> Vysoka kladna hodnota (3) = leva hrana svisle cary")
print("> Vysoka zaporna hodnota (-3) = prava hrana svisle cary")
print("> Nulove hodnoty = zadna hrana")

---

## 3. Galerie filtru (kernelu)

Ruzne filtry hledaji ruzne vzory. Podivejme se na nejpouzvanejsi:

In [ ]:
# Definice zakladnich filtru

# 1. Detektor horizontalnich hran
filtr_horizontalni = np.array([
    [ 1,  1,  1],
    [ 0,  0,  0],
    [-1, -1, -1]
])

# 2. Detektor vertikalnich hran
filtr_vertikalni = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

# 3. Sobel filtr (lepsi detekce hran)
sobel_x = np.array([
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]
])

sobel_y = np.array([
    [-1, -2, -1],
    [ 0,  0,  0],
    [ 1,  2,  1]
])

# 4. Filtr pro zaostreni (sharpen)
filtr_sharpen = np.array([
    [ 0, -1,  0],
    [-1,  5, -1],
    [ 0, -1,  0]
])

# 5. Filtr pro rozmazani (blur/box blur)
filtr_blur = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]
]) / 9.0  # Deleni 9 pro normalizaci

# 6. Gaussovsky blur (jemnejsi rozmazani)
filtr_gauss = np.array([
    [1, 2, 1],
    [2, 4, 2],
    [1, 2, 1]
]) / 16.0

# Vizualizace vsech filtru
filtry = [
    (filtr_horizontalni, 'Horizontalni hrany'),
    (filtr_vertikalni, 'Vertikalni hrany'),
    (sobel_x, 'Sobel X'),
    (sobel_y, 'Sobel Y'),
    (filtr_sharpen, 'Zaostreni'),
    (filtr_gauss, 'Gaussovsky blur')
]

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for ax, (filtr, nazev) in zip(axes, filtry):
    im = ax.imshow(filtr, cmap='RdBu', vmin=-2, vmax=2)
    ax.set_title(nazev, fontsize=12)
    for i in range(3):
        for j in range(3):
            val = filtr[i, j]
            ax.text(j, i, f'{val:.1f}' if val != int(val) else str(int(val)), 
                   ha='center', va='center', fontsize=10)

plt.suptitle('Galerie konvolucnich filtru', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---

## 4. Prakticky projekt: Filtry na realnem obrazku

Ted aplikujeme filtry na skutecny obrazek a uvidime jejich efekt:

In [ ]:
# Nacteni testovacich obrazku ze skimage
obrazek_kamera = data.camera()  # Slavny obrazek "kameramana"
obrazek_astronaut = rgb2gray(data.astronaut())  # Astronaut v odstinech sedi

# Zobrazime oba obrazky
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(obrazek_kamera, cmap='gray')
axes[0].set_title('Cameraman (klasicky testovaci obrazek)', fontsize=12)
axes[0].axis('off')

axes[1].imshow(obrazek_astronaut, cmap='gray')
axes[1].set_title('Astronaut (odstiny sedi)', fontsize=12)
axes[1].axis('off')

plt.suptitle('Testovaci obrazky pro konvoluci', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Aplikace vsech filtru na obrazek kameramana
obrazek = obrazek_kamera.astype(float) / 255.0  # Normalizace na 0-1

# Provedeni konvoluce pro kazdy filtr
vysledky = {}
for filtr, nazev in filtry:
    vysledky[nazev] = convolve2d(obrazek, filtr, mode='same', boundary='symm')

# Vizualizace vysledku
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

# Puvodni obrazek
axes[0, 0].imshow(obrazek, cmap='gray')
axes[0, 0].set_title('ORIGINAL', fontsize=12, fontweight='bold')
axes[0, 0].axis('off')

# Vysledky filtru
pozice = [(0,1), (0,2), (0,3), (1,0), (1,1), (1,2)]
for (row, col), (nazev, vysledek) in zip(pozice, vysledky.items()):
    # Pro detektory hran pouzijeme absolutni hodnotu
    if 'hran' in nazev.lower() or 'sobel' in nazev.lower():
        axes[row, col].imshow(np.abs(vysledek), cmap='gray')
    else:
        axes[row, col].imshow(vysledek, cmap='gray')
    axes[row, col].set_title(nazev, fontsize=11)
    axes[row, col].axis('off')

# Skryt posledni prazdny subplot
axes[1, 3].axis('off')

plt.suptitle('Efekt ruznych konvolucnich filtru', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### Porovnani detekce hran

In [ ]:
# Detailni porovnani detekce hran
obrazek = obrazek_kamera.astype(float) / 255.0

# Aplikace Sobel filtru v obou smerech
hrany_x = convolve2d(obrazek, sobel_x, mode='same', boundary='symm')
hrany_y = convolve2d(obrazek, sobel_y, mode='same', boundary='symm')

# Kombinovana magnituda hran
hrany_kombinovane = np.sqrt(hrany_x**2 + hrany_y**2)

# Vizualizace
fig, axes = plt.subplots(2, 2, figsize=(12, 12))

axes[0, 0].imshow(obrazek, cmap='gray')
axes[0, 0].set_title('Puvodni obrazek', fontsize=14)
axes[0, 0].axis('off')

axes[0, 1].imshow(np.abs(hrany_x), cmap='gray')
axes[0, 1].set_title('Sobel X (vertikalni hrany)', fontsize=14)
axes[0, 1].axis('off')

axes[1, 0].imshow(np.abs(hrany_y), cmap='gray')
axes[1, 0].set_title('Sobel Y (horizontalni hrany)', fontsize=14)
axes[1, 0].axis('off')

axes[1, 1].imshow(hrany_kombinovane, cmap='gray')
axes[1, 1].set_title('Kombinovana magnituda hran', fontsize=14)
axes[1, 1].axis('off')

plt.suptitle('Detekce hran pomoci Sobel operatoru', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

print("Sobel X najde vertikalni hrany (zmeny zleva doprava)")
print("Sobel Y najde horizontalni hrany (zmeny shora dolu)")
print("Kombinace dava kompletni mapu vsech hran v obrazku")

---

## 5. Vytvorte si vlastni filtr!

Experimentujte s vlastnimi filtry a sledujte, co delaji:

In [ ]:
# Vlastni filtr - zkuste menit hodnoty!
muj_filtr = np.array([
    [-1, -1, -1],
    [-1,  8, -1],
    [-1, -1, -1]
])  # Toto je Laplacian filtr - detekuje vsechny hrany

# Aplikace na obrazek
obrazek = obrazek_kamera.astype(float) / 255.0
vysledek_muj = convolve2d(obrazek, muj_filtr, mode='same', boundary='symm')

# Vizualizace
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(obrazek, cmap='gray')
axes[0].set_title('Puvodni', fontsize=14)
axes[0].axis('off')

im = axes[1].imshow(muj_filtr, cmap='RdBu', vmin=-2, vmax=8)
axes[1].set_title('Vas filtr (Laplacian)', fontsize=14)
for i in range(3):
    for j in range(3):
        axes[1].text(j, i, str(muj_filtr[i, j]), ha='center', va='center', fontsize=12)
plt.colorbar(im, ax=axes[1])

axes[2].imshow(np.abs(vysledek_muj), cmap='gray')
axes[2].set_title('Vysledek (detekce vsech hran)', fontsize=14)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\nTipy pro experimentovani:")
print("- Zmente stredni hodnotu (8) na vetsi/mensi cislo")
print("- Zkuste ruzne kombinace kladnych a zapornych cisel")
print("- Filtr se souctem 0 = detekce hran")
print("- Filtr se souctem 1 = zachovani jasu (blur, sharpen)")

---

## 6. Uvod do konvolucnich neuonovych siti (CNN)

A ted to nejdulezitejsi - jak to souvisi s AI?

**Konvolucni neuronova sit (CNN)** je specialni typ site, ktera:
1. Pouziva konvolucni vrstvy misto plne propojenych
2. **Filtry se UCI SAMY** behem treninku!
3. Hierarchicky se uci od jednoduchych vzoru ke slozitym

In [ ]:
# Vizualizace hierarchie uceni v CNN
fig, ax = plt.subplots(figsize=(14, 6))

# Popis vrstev CNN
vrstvy = [
    ('Vstup', 'Obrazek\n(pixely)', 'lightblue'),
    ('Vrstva 1', 'Hrany\nCary\nBarvy', 'lightgreen'),
    ('Vrstva 2', 'Textury\nRohy\nTvary', 'lightyellow'),
    ('Vrstva 3', 'Casti objektu\n(oci, nos, kola)', 'lightsalmon'),
    ('Vrstva 4', 'Cele objekty\n(obliceje, auta)', 'lightpink'),
    ('Vystup', 'Klasifikace\n(pes/kocka)', 'lavender')
]

# Kresleni vrstev
for i, (nazev, popis, barva) in enumerate(vrstvy):
    x = i * 2.2
    rect = plt.Rectangle((x, 0), 1.8, 3, facecolor=barva, edgecolor='black', linewidth=2)
    ax.add_patch(rect)
    ax.text(x + 0.9, 3.3, nazev, ha='center', va='bottom', fontsize=11, fontweight='bold')
    ax.text(x + 0.9, 1.5, popis, ha='center', va='center', fontsize=9)
    
    # Sipky mezi vrstvami
    if i < len(vrstvy) - 1:
        ax.annotate('', xy=(x + 2.2, 1.5), xytext=(x + 1.8, 1.5),
                   arrowprops=dict(arrowstyle='->', color='gray', lw=2))

ax.set_xlim(-0.5, 13.5)
ax.set_ylim(-0.5, 4.5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Hierarchie uceni v CNN - od pixelu k rozpoznani objektu', 
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

In [ ]:
# Simulace - co by se mohla CNN naucit
# (Pro skutecne uceni potrebujeme PyTorch/TensorFlow, to prijde v dalsich kapitolach)

print("=" * 60)
print("CO SE CNN UCI AUTOMATICKY:")
print("=" * 60)
print()
print("1. VRSTVA - Zakladni filtry:")
print("   - Detektory hran (horizontalni, vertikalni, diagonalni)")
print("   - Detektory barev")
print("   - Detektory kontrastu")
print()
print("2. VRSTVA - Kombinace zakladnich vzoru:")
print("   - Rohy a zakrivenirany")
print("   - Jednoduche textury")
print("   - Male tvary")
print()
print("3. VRSTVA - Casti objektu:")
print("   - Oci, nosy, usi (pro obliceje)")
print("   - Kola, okna (pro auta)")
print("   - Listy, vetev (pro rostliny)")
print()
print("4. VRSTVA - Cele objekty:")
print("   - Cely obliceje")
print("   - Cela auta")
print("   - Cele zvirata")
print()
print("KOUZLO: Hodnoty filtru se UCI SAMY pomoci zpetne propagace!")
print("Neprogramujeme je rucne - sit je OBJEVI sama z dat.")

---

## 7. Pooling - Zmensovani map priznaku

CNN pouziva jeste jednu dulezitou operaci - **pooling** (sdružování). Ta zmenšuje velikost map a vybírá nejdůležitější informace:

In [ ]:
def max_pooling(obrazek, velikost_okna=2):
    """Jednoducha implementace max poolingu."""
    vyska, sirka = obrazek.shape
    nova_vyska = vyska // velikost_okna
    nova_sirka = sirka // velikost_okna
    
    vystup = np.zeros((nova_vyska, nova_sirka))
    
    for i in range(nova_vyska):
        for j in range(nova_sirka):
            # Vybereme maximum z okna 2x2
            okno = obrazek[i*2:(i+1)*2, j*2:(j+1)*2]
            vystup[i, j] = np.max(okno)
    
    return vystup

# Priklad poolingu
priklad = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

pooled = max_pooling(priklad)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].imshow(priklad, cmap='viridis')
axes[0].set_title('Puvodni (4x4)', fontsize=14)
for i in range(4):
    for j in range(4):
        axes[0].text(j, i, str(priklad[i, j]), ha='center', va='center', 
                    fontsize=12, color='white')

axes[1].imshow(pooled, cmap='viridis')
axes[1].set_title('Po Max Pooling (2x2)', fontsize=14)
for i in range(2):
    for j in range(2):
        axes[1].text(j, i, str(int(pooled[i, j])), ha='center', va='center', 
                    fontsize=12, color='white')

plt.suptitle('Max Pooling - vybirame maximum z kazdeho 2x2 okna', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Max Pooling:")
print(f"- Z okna [1,2,5,6] vybere maximum: 6")
print(f"- Z okna [3,4,7,8] vybere maximum: 8")
print(f"- Zmensuje velikost, ale zachovava dulezite informace")

In [ ]:
# Pooling na realnem obrazku
obrazek = obrazek_kamera.astype(float)

# Aplikace vice urovni poolingu
pool1 = max_pooling(obrazek, 2)  # 512x512 -> 256x256
pool2 = max_pooling(pool1, 2)    # 256x256 -> 128x128
pool3 = max_pooling(pool2, 2)    # 128x128 -> 64x64
pool4 = max_pooling(pool3, 2)    # 64x64 -> 32x32

fig, axes = plt.subplots(1, 5, figsize=(16, 4))

obrazky = [obrazek, pool1, pool2, pool3, pool4]
nazvy = ['Original\n512x512', 'Pool 1x\n256x256', 'Pool 2x\n128x128', 
         'Pool 3x\n64x64', 'Pool 4x\n32x32']

for ax, obr, nazev in zip(axes, obrazky, nazvy):
    ax.imshow(obr, cmap='gray')
    ax.set_title(nazev, fontsize=11)
    ax.axis('off')

plt.suptitle('Postupne zmensovani obrazku pomoci Max Pooling', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Pooling v CNN:")
print("- Zmensuje prostorove rozmery (rychlejsi vypocet)")
print("- Zaveduje invarianci vuci posunum (objekt muze byt mekde)")
print("- Zvysuje 'receptivni pole' neuronu (vidi vetsi oblast)")

---

## 8. Mini-kviz

Otestujte sve znalosti:

In [ ]:
# Mini-kviz o konvoluci
print("=" * 60)
print("KVIZ: KONVOLUCE A CNN")
print("=" * 60)
print()

otazky = [
    {
        "otazka": "1. Co je konvolucni filtr (kernel)?",
        "moznosti": [
            "a) Velka neuronova sit",
            "b) Mala matice cisel, ktera hleda vzory v obrazku",
            "c) Algoritmus pro trenovani",
            "d) Typ aktivacni funkce"
        ],
        "spravna": "b",
        "vysvetleni": "Filtr je mala matice (typicky 3x3), ktera se posouva pres obrazek a hleda specificke vzory."
    },
    {
        "otazka": "2. Co dela Sobel filtr?",
        "moznosti": [
            "a) Rozmazava obrazek",
            "b) Meni barvy",
            "c) Detekuje hrany",
            "d) Zvetuje obrazek"
        ],
        "spravna": "c",
        "vysvetleni": "Sobel filtr je klasicky detektor hran - najde mista, kde se jas rychle meni."
    },
    {
        "otazka": "3. Co je 'mapa priznaku' (feature map)?",
        "moznosti": [
            "a) Seznam vsech pixelu",
            "b) Vysledek aplikace filtru na obrazek",
            "c) Typ neuronove site",
            "d) Databaze obrazku"
        ],
        "spravna": "b",
        "vysvetleni": "Mapa priznaku je vysledny obrazek po aplikaci konvolucniho filtru - ukazuje, kde filtr nasel hledany vzor."
    },
    {
        "otazka": "4. Co je hlavni vyhoda CNN oproti klasickym sitim pro obrazky?",
        "moznosti": [
            "a) Je rychlejsi",
            "b) Automaticky se uci hierarchii vizualnich priznaku",
            "c) Potrebuje mene dat",
            "d) Je jednodussi na implementaci"
        ],
        "spravna": "b",
        "vysvetleni": "CNN se sama nauci detektory od jednoduchych hran az po slozite objekty - neprogramujeme filtry rucne!"
    },
    {
        "otazka": "5. Co dela Max Pooling?",
        "moznosti": [
            "a) Zvetsuje obrazek",
            "b) Zmensuje prostorove rozmery a vybira nejsilnejsi signaly",
            "c) Pridava barvy",
            "d) Trenuje vahy site"
        ],
        "spravna": "b",
        "vysvetleni": "Max Pooling zmensuje velikost mapy priznaku tim, ze vybira maximum z kazdeho okna."
    }
]

skore = 0
for q in otazky:
    print(q["otazka"])
    for m in q["moznosti"]:
        print(f"   {m}")
    print(f"   Spravna odpoved: {q['spravna']}")
    print(f"   > {q['vysvetleni']}")
    print()

---

## 9. Shrnuti kapitoly

V teto kapitole jsme se naucili:

| Koncept | Popis |
|---------|-------|
| **Konvoluce** | Matematicka operace - posuvani filtru pres obrazek |
| **Filtr/Kernel** | Mala matice cisel, ktera hleda specificke vzory |
| **Mapa priznaku** | Vysledek konvoluce - kde filtr nasel hledany vzor |
| **Detekce hran** | Sobel, Laplacian - filtry pro hledani hran |
| **Blur/Sharpen** | Filtry pro rozmazani nebo zaostreni |
| **CNN** | Sit, ktera se SAMA uci filtry z dat |
| **Pooling** | Zmensovani map priznaku (Max, Average) |

### Klicove poznatky:
- Konvoluce je zaklad pocitacoveho videni
- Ruzne filtry hledaji ruzne vzory (hrany, textury, tvary)
- CNN se automaticky uci hierarchii filtru od jednoduchych k slozitym
- Pooling zmensuje velikost a zvysuje odolnost vuci posunum

---

## 10. Vyzva pro vas

Zkuste tyto experimenty:

In [ ]:
# VYZVA 1: Vytvorte filtr pro diagonalni hrany
# Tip: kombinujte kladna a zaporna cisla po diagonale

filtr_diagonalni = np.array([
    [2, 1, 0],
    [1, 0, -1],
    [0, -1, -2]
])

# Vase reseni:
# ...

print("VYZVA 1: Experimentujte s filtrem pro diagonalni hrany")
print("Zkuste zmenit hodnoty a pozorovat vysledek!")

In [ ]:
# VYZVA 2: Porovnejte efekt ruznych velikosti rozmazovaciho filtru
# Vytvorte blur filtry 3x3, 5x5 a 7x7

obrazek = obrazek_kamera.astype(float) / 255.0

blur_3x3 = np.ones((3, 3)) / 9
blur_5x5 = np.ones((5, 5)) / 25
blur_7x7 = np.ones((7, 7)) / 49

# Aplikujte filtry a porovnejte vysledky
# ...

print("VYZVA 2: Porovnejte ruzne velikosti blur filtru")
print("Vetsi filtr = silnejsi rozmazani")

In [ ]:
# VYZVA 3: Zkombinujte detekci hran se zaostrenim
# Co se stane, kdyz nejdriv detekujeme hrany a pak zaostrime?

print("VYZVA 3: Retezeni filtru")
print("Aplikujte filtry postupne a pozorujte kombinovany efekt!")
print()
print("Toto je zaklad toho, jak CNN funguje - kazda vrstva")
print("aplikuje sve filtry na vysledky predchozi vrstvy!")

---

## Dalsi kroky

V pristich kapitolach se naucime:
- Jak postavit skutecnou CNN v PyTorchu
- Jak trenovat CNN na rozpoznavani obrazku
- Transfer learning - vyuziti predtrenovanych modelu

**Konvoluce je zaklad moderniho pocitacoveho videni - od rozpoznavani obliceju po samoridici auta!**